<a href="https://colab.research.google.com/github/Maks6666/ML/blob/main/recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Завдання

* Створіть Reader
* Створіть датасет та розділіть його на тренувальні та тестові дані
* Виберіть метрики для поріняння якості моделей
* На основі метрик виберіть найкращу модель
* Використайте `optuna` для підбору найкращих параметрів



In [3]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module7/data/Blog%20Ratings.csv")

df.head()

,blog_id,userId,ratings
0,9025,11,3.5
1,9320,11,5.0
2,9246,11,3.5
3,9431,11,5.0
4,875,11,2.0


In [1]:
!pip install -q surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
df.describe()

,blog_id,userId,ratings
count,200140.000000,200140.000000,200140.000000
mean,5652.533621,2545.710158,3.117468
std,2970.685946,1446.195478,1.768113
min,1.000000,10.000000,0.500000
25%,2906.000000,1314.000000,2.000000
50%,5994.000000,2552.000000,3.500000
75%,8510.000000,3795.000000,5.000000
max,9755.000000,5010.000000,5.000000


In [52]:

from surprise import Dataset, SVD, Reader

reader = Reader(rating_scale=(0, 5))

data = Dataset.load_from_df(df[["blog_id", "userId", "ratings"]], reader)

In [8]:
data

In [46]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, train_size=0.8)

In [10]:
trainset

In [47]:
from surprise import BaselineOnly

bsl_options = {'method': 'als',
               'reg_u': 0.0001,
               'reg_i': 0.0001,
               ' n_epochs': 100}

model_1 = BaselineOnly(bsl_options=bsl_options)
model_1.fit(trainset)



Estimating biases using als...


In [48]:
model_1.predict(uid=875,
             iid=11)

# 2

Prediction(uid=875, iid=11, r_ui=None, est=4.4874472958780816, details={'was_impossible': False})

In [64]:

from surprise import BaselineOnly, accuracy

preds = model_1.test(testset)

print(f"mae = {accuracy.mae(preds, verbose=False)}")
print(f"mse = {accuracy.mse(preds, verbose=False)}")
print(f"rmse= {accuracy.rmse(preds, verbose=False)}")
print(f"fcp = {accuracy.fcp(preds, verbose=False)}")

mae = 1.6367883197051192
mse = 3.4283274168565443
rmse= 1.8515743076788855
fcp = 0.5052565379029572


In [56]:

from surprise import SVD

model_2 = SVD(n_factors=100,
           n_epochs=50,
           lr_all = 0.01,
           reg_all = 0.001)

model_2.fit(trainset)

In [54]:
model_2.predict(uid=875,
             iid=11)


# 2

Prediction(uid=875, iid=11, r_ui=None, est=2.1662085580179054, details={'was_impossible': False})

In [65]:

from surprise import BaselineOnly, accuracy

preds = model_2.test(testset)

print(f"mae = {accuracy.mae(preds, verbose=False)}")
print(f"mse = {accuracy.mse(preds, verbose=False)}")
print(f"rmse= {accuracy.rmse(preds, verbose=False)}")
print(f"fcp = {accuracy.fcp(preds, verbose=False)}")

mae = 1.6787598897471598
mse = 3.775766528677716
rmse= 1.94313317317103
fcp = 0.4980003323370906


In [59]:
from surprise import KNNBasic

model_3 = KNNBasic(k=120,
                min_k=4,
                sim_options={'name': 'pearson',
                             'user_based': True},
                shrinkage = 90)

model_3.fit(trainset)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [66]:

from surprise import BaselineOnly, accuracy

preds = model_3.test(testset)

print(f"mae = {accuracy.mae(preds, verbose=False)}")
print(f"mse = {accuracy.mse(preds, verbose=False)}")
print(f"rmse= {accuracy.rmse(preds, verbose=False)}")
print(f"fcp = {accuracy.fcp(preds, verbose=False)}")

mae = 1.6088767564276851
mse = 3.115249993873017
rmse= 1.7650070804030835
fcp = 0.36023951950768257


In [60]:
model_3.predict(uid=875,
             iid=11)

Prediction(uid=875, iid=11, r_ui=None, est=3.1165402967922455, details={'was_impossible': True, 'reason': 'Not enough neighbors.'})

In [61]:
from surprise import CoClustering

model_4 = CoClustering(n_cltr_u = 3,
                    n_cltr_i = 20,
                    n_epochs = 90)

model_4.fit(trainset)

In [67]:

from surprise import BaselineOnly, accuracy

preds = model_4.test(testset)

print(f"mae = {accuracy.mae(preds, verbose=False)}")
print(f"mse = {accuracy.mse(preds, verbose=False)}")
print(f"rmse= {accuracy.rmse(preds, verbose=False)}")
print(f"fcp = {accuracy.fcp(preds, verbose=False)}")

mae = 1.7143468228402192
mse = 4.0047139822289735
rmse= 2.0011781485487425
fcp = 0.5051264264337502


In [38]:
model_4.predict(uid=875,
             iid=11)

Prediction(uid=875, iid=11, r_ui=None, est=4.350582814471027, details={'was_impossible': False})